# 目标：工作站管理平台设计

工作站的主要功能是管理、控制所有终端设备。包括查看终端设备的工作状态，配置设备与病人之间的绑定关系，查看病人/设备历史数据。查看设备的实时数据等。

工作站所有有几个部分组成：
* 人机交互界面：给用户提供设备状态或设备波形的界面，提供对数据修改的窗口。
* 设备状态与数据内容：设备状态与本地数据的缓存，数据的增、改、删都会发送在这里
* 人机交互和本地数据之间的协调与控制，以及与远程数据库之间的交互：这个部分处于设备数据和人机交互之间，界面上的按钮对设备数据的维护将通过此部分实现，而不直接访问。这样的目的是让数据和界面解耦。同时此部分会与数据库进行交互，承担这整个系统的对外接口。

根据分析工作站的核心是设备的数据，所有的操作都是针对这些数据的。例如查看当前设备列表以及设备状态，例如增加设备、删除设备、修改设备的当前用户等都是在访问这些数据。所以我们会把系统拆分为三大类：
* 显示部分（显示）：负责数据内容的显示
* 数据部分（模型）：创建以设备数据为核心的模型，存储设备数据。这里把数据抽象为模型。
* 控制部分（控制）：负责逻辑部分的处理，为显示部分和数据部分提供桥梁当数据改变时及时更新显示，当显示界面修改数据时把修改后的结果写回数据部分，同时实现访问外部数据接口。

此种设计方式是典型的[MVC模式](https://baike.baidu.com/item/MVC%E6%A1%86%E6%9E%B6/9241230?fromtitle=mvc&fromid=85990&fr=aladdin)


效果图：
![](./image/qt_workstation_table_demo.png)


# 参考

https://doc.qt.io/qt-5/model-view-programming.html#model-classes  
http://c.biancheng.net/view/1864.html  
https://blog.csdn.net/naughfy/article/details/79834849  
http://codingdict.com/sources/py/PyQt5.QtGui/10121.html
https://www.cnblogs.com/fuqia/p/8908182.html

# 创建模型

与终端设备相关的数据有：

|数据内容|格式|描述|
|:----|:----|:----|
|编号|32位数字|设备唯一的识别码|
|名称|字符串|描述设备的功能|
|启用日期|日期|记录设备何时开始使用|
|在线状态|布尔型，在线/离线|设备是否在线|
|用户|结构体|包含编号、姓名、性别、年龄、电话|
|负责人|结构图|维护此设备的编号、姓名、电话|

**查看数据**：调用数据查看界面  
**查看波形**：调用历史波形/实时波形界面，与设备端界面相同


## 设计MonitorDevInfo类

* 数据存储的单元
* 提供数据调用方法，这里可直接使用`user.name`访问病人姓名

In [1]:
# 创建模型数据结构

class PatientInfo():
    def __init__(self, id = 20200701, name = "小宝", sex = "男", age = 10, tel = "13734563456"):
        self.id = id
        self.name = name
        self.sex = sex
        self.age = age
        self.tel = tel

class OwnerInfo():
    def __init__(id = 123456, name="小花", tel = "13734563456"):
        self.ownerId = id
        self.ownerName = name
        self.ownerTel = tel
        
class MonitorDevInfo():
    def __init__(self,id = 123456789, name = "DEV_001", user = None, dept = "住院部", 
                 owner = None):
        self.id = id
        self.name = name
        self.netStatus = False
        self.user = user
        self.owner = owner

## 设计监护设备数据模型

* 使用标准模型类作为成员变量
* 使用MonitorDevInfo作为最小数据单元

作为模型，需要提供数据的添加、删除、修改等功能

In [2]:
# 创建模型

from PyQt5.QtGui import QStandardItemModel, QStandardItem
from PyQt5.QtWidgets import QTableView
from PyQt5.QtCore import Qt, QModelIndex

class MonitorDevModel(QStandardItemModel):
    
    def __init__(self, parent= None):
        super(MonitorDevModel, self).__init__(parent)
        self.itemModel = QStandardItemModel()
        self.itemModel.setHorizontalHeaderLabels(['设备编号', '设备型号', '网络状态', '病人编码', '病人姓名'])
    
    def setView(self, view: QTableView):
        view.setModel(self.itemModel)

    def addData(self, devInfo: MonitorDevInfo):
        # 创建QStandardItem对象
        itemDevId = QStandardItem()
        itemDevName = QStandardItem()
        itemDevNetStatus = QStandardItem()
        itemDevUserId = QStandardItem()
        itemDevUserName = QStandardItem()
        itemDevUserSex = QStandardItem()
        itemDevUserAge = QStandardItem()
        itemDevUserTel = QStandardItem()
        itemDevOwnerId = QStandardItem()
        itemDevOwnerName = QStandardItem()
        itemDevOwnerTel = QStandardItem()
        
        itemDevId.setData(devInfo.id, Qt.DisplayRole)
        itemDevName.setData(devInfo.name, Qt.DisplayRole)
        itemDevUserId.setData(devInfo.user.id, Qt.DisplayRole)
        itemDevUserName.setData(devInfo.user.name, Qt.DisplayRole)
        # todo other 
            
        # 把创建的对象添加到列表中
        self.itemModel.insertRow(self.rowCount(), [itemDevId, itemDevName, itemDevNetStatus, itemDevUserId, itemDevUserName, itemDevUserSex, itemDevUserAge,
                             itemDevUserTel, itemDevOwnerId, itemDevOwnerName, itemDevOwnerTel])
    
    def getData(self, row):
        devInfo = MonitorDevInfo()
        colCount = self.columnCount()
        
        if row >= 0 and row < self.rowCount():
            for col in range(colCount):
                index = self.itemModel.index(row, col, QModelIndex())
                value = index.data(Qt.DisplayRole)
                
                if col == 0:
                    devInfo.id = value
                # todo
                
        return devInfo
            
    def setData(self, row, devInfo: MonitorDevInfo):
        # todo
        return True
        
    def removeRow(self, row):
        self.itemModel.removeRow(row)
        return True
    
    def clearData(self):
        # todo
        return True
    
    def rowCount(self):
        return self.itemModel.rowCount()

    def colCount(self):
        return self.itemModel.columnCount()

# 创建显示界面

In [3]:
from PyQt5.QtWidgets import QApplication, QWidget, QTableView, QMainWindow, QVBoxLayout

class WindowMain(QWidget):
    def __init__(self, parent = None):
        super(WindowMain, self).__init__(parent)
        self.layout = QVBoxLayout()

        self.tableView = QTableView()

        self.layout.addWidget(self.tableView)
        self.setLayout(self.layout)
        self.resize(1500, 800)
        self.setWindowTitle("云监护-工作站")
    
    # todo

# 创建控制器

In [4]:
class Control():
    def __init__(self):
        self.dataModel = MonitorDevModel()
        self.win = WindowMain()
        
        self.dataModel.setView(self.win.tableView)
        
        # demo: 增加10行数据
        for i in range(10):
            info = MonitorDevInfo(id = i, name="ECG_%s"%i, user= PatientInfo(id = i*10, name = "张三"))
            self.dataModel.addData(info)
    
    # todo

In [5]:
if __name__ == "__main__":
    app = QApplication([])
    
    main = Control()
    main.win.show()
    
    app.exec_()